### Свертка через перемножение матриц

Проводить вычисления при помощи вложенных циклов малоэффективно. Но операцию свертки можно реализовать через матричное умножение, которое очень эффективно выполняется на GPU. Идею того, как это происходит, можно продемонстрировать на примере 1D-свертки. Будем сворачивать 1D-сигнал `input_1d` с 1D-фильтром `kernel_1d`. Воспользуемся операцией 1D-свертки из библиотеки NumPy:

In [ ]:
import numpy as np


input_1d = np.array([1, 2, 4, 5, 3, 0])
kernel_1d = np.array([5, 4])

out = np.convolve(
    input_1d,
    np.flip(kernel_1d),  # because real conv reverse order of elements
    mode="valid",
)

# 5*1 + 4*2 = 5 + 8   = 13
# 5*2 + 4*4 = 10 + 16 = 26
# 5*4 + 4*5 = 20 + 20 = 40
# 5*5 + 4*3 = 25 + 12 = 37
# 5*3 + 4*0 = 15 + 0  = 15

print(out)

[13 26 40 37 15]


После того, как мы отменили "переворот", выполнив `np.flip` и превратив свертку в взаимнокорреляционную функцию, результат совпал с ожидаемым. Восстановим последовательность действий по шагам:

**Шаг 1.**

<img src ="https://ml.gan4x4.ru/msu/dev-2.0/L06/out/matrix_conv1.png" width="600">

Перемножаем первые два входа на значения из ядра фильтра и суммируем их.

**Шаг 2**

<img src ="https://ml.gan4x4.ru/msu/dev-2.0/L06/out/matrix_conv2.png" width="600">

Сдвигаем фильтр и повторяем операцию.

...

**Шаг N**

<img src ="https://ml.gan4x4.ru/msu/dev-2.0/L06/out/matrix_conv3.png" width="600">

Если мы дополним фильтр нулями вот таким образом:

<img src ="https://ml.gan4x4.ru/msu/dev-2.0/L06/out/matrix_conv4.png" width="600">

То результат не поменяется.

Видно, что если скомбинировать дополненные нулями ядра в матрицу и транспонировать ее, то мы получим тот же результат, скалярно умножив входной вектор на эту матрицу.

In [ ]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.diags.html
from scipy.sparse import diags


# Alternative to sparse_matrix = diags([5,4],[0,1],shape=(5, 6)).toarray()
output_shape = len(input_1d) - len(kernel_1d) + 1
sparse_matrix = diags(
    kernel_1d, [0, 1], shape=(output_shape, len(input_1d))  # diagonals positions
).toarray()

print(sparse_matrix)

[[5. 4. 0. 0. 0. 0.]
 [0. 5. 4. 0. 0. 0.]
 [0. 0. 5. 4. 0. 0.]
 [0. 0. 0. 5. 4. 0.]
 [0. 0. 0. 0. 5. 4.]]


In [ ]:
out2 = sparse_matrix.dot(input_1d)
print("Result ", out2)

Result  [13. 26. 40. 37. 15.]


<img src ="https://ml.gan4x4.ru/msu/dev-2.0/L06/out/matrix_conv5.png" width="900">

Для двумерной свертки действует похожая логика, подробне можно прочесть здесь:

[2D Convolution as a Matrix-Matrix Multiplication](https://www.baeldung.com/cs/convolution-matrix-multiplication)

И при получении градиентов это тоже работает:

[Forward and Backward Convolution Passes as Matrix Multiplication](https://danieltakeshi.github.io/2019/03/09/conv-matmul/)


